# Data Science Project: Planning Stage (Individual Portion)

### Data Description:

A UBC research group is collecting data on how people play videogames. Below is a summary of the datasets:

##### Sessions Data: sessions.csv

Observations(rows):1535  
Variables(columns):5

##### Variable Information:
|Variable Name| Type    | Description|
|------------|----------|--------|
|hashedEmail |Character |Player's unique hashed anonymous email(same as players.csv)|
|start_time  |Character |Game session start for player (human-readable)|
|end_time    |Character |Game session end for player (human-readable)|
|original_start_time|Double |Original session start time (server time stamp)| 
|original_end_time|Double |Original session end time (server time stamp)|

##### Additional Details:
- No session duration included(must be computed).
- Date and time included in start/end times are not useable for computation currently and must be converted allow for the session time to be calculated.
- Inconsistancies due to time zone variations.
- Sessions may include missing or inconsistant data.
- Potential errors(mismatched/incorrect) in email imputs.

  
##### Player Data: players.csv

Observations(rows): 196  
Variables(columns): 7

##### Variable Information:
|Variable Name| Type    | Description|
|------------|----------|--------|
|experience  |Character |Player's self reported experience level|
|subscribe   |Logical   |Whether the player subscribed to a game-related newsletter (True/False)|
|hashedEmail |Character |Player's unique hashed anonymous email|
|played_hours|Double    |Player's total hours in the game|
|name        |Character |Player's in game name|
|gender      |Character |Player self reported gender| 
|age         |Double    |Player age in years (no decimals)|

##### Additional Details:
- Experience level is self reported and may be biased resulting in inacurate representation. 
- Sampling bias may affect results(may not represent general population).
- Self reported variables may be biased. 
- Missing values may be present affecting analysis.
- Age and played_hours may have outliers causing the results to be skewed.


### Questions:  
##### Broad question:  
What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?  

##### Specific question:  
Can a players total playtime(played_hours), experience level (experience), number of sessions(number_of_sessions), and mean session length(mean_session_length) predict whether they will subscribe to the game-related newsletter(subscribe)?  

### Exploratory Data Analysis and Visualization:

In [ ]:
# Loading Libraries
library(tidyverse)
library(tidymodels)
library(dplyr)
library(ggplot2)
library(RColorBrewer)
library(repr)
library(lubridate)
library(readr)

#limiting dataframe outputs to 6 rows
options(repr.matrix.max.rows = 6)

In [ ]:
#Load the Data

players<- read_csv(
        "https://raw.githubusercontent.com/ajones200/dsci100_individual/refs/heads/main/players.csv") 
players

sessions<- read_csv(
        "https://raw.githubusercontent.com/ajones200/dsci100_individual/refs/heads/main/sessions.csv")
sessions

In [ ]:
#View Players Dataset information
cat("Players Dataset:\n","Rows:", nrow(players),"\n","Columns:", ncol(players))
cat("\n")
cat("\nPlayers Dataset Variable types:\n")
sapply(players, typeof)

#Check number of missing variables
cat("\nPlayers dataset missing variables:\n") 
    print(colSums(is.na(players)))

#Look at all unique outcomes for the quatitative variables with limited results (gender and experience)
#These are self assigned variables
cat("Possible unique outcomes for some catagorical variables\n")
players|>
select(experience, gender)|>
map(unique)

In [ ]:
#View Sessions Dataset information
cat("Sessions Dataset:\n","Rows:", nrow(sessions),"\n","Columns:", ncol(sessions))
cat("\n")
cat("\nSessions Dataset Variable types:\n")
sapply(sessions, typeof)

#Check number of missing variables
cat("\nSessions Dataset missing Variables:\n") 
    print(colSums(is.na(sessions)))

In [ ]:
#Calculate the mean of each quantitative variable from the players dataset
cat("Mean of Quantitative Variables in Players Dataset:\n")

numeric_players<- players[sapply(players, is.numeric)]

players_table_mean<-colMeans(numeric_players, na.rm= TRUE)
    
players_table_mean

In [ ]:
#Data Wrangling
#Remove NA values from the datasets
players_clean<- na.omit(players)
sessions_clean<- na.omit(sessions)

#convert start and end times (character strings) to an object that can be used to compute the duration of each session in minutes.
sessions_time_mutate<- sessions_clean|>
    mutate(
        start= dmy_hm(start_time, tz = "UTC"), 
        end= dmy_hm(end_time, tz = "UTC"),
        duration_mins = as.numeric(difftime(end, start, units= "mins")))
#When wanting to use the mean session length for computation must probably convert to hours, standardize or change the hours_played to minutes.

#calculate mean session time and number of sessions
sessions_organized<- sessions_time_mutate|>
    group_by(hashedEmail)|>
    summarize(
        number_of_sessions = n(),
        mean_session_length_mins = mean (duration_mins, na.rm=TRUE),
        )

In [ ]:
# Merge the datasets
players_merged<- players_clean|>
    left_join(sessions_organized, by = "hashedEmail")|>
    filter(!is.na(number_of_sessions), !is.na(mean_session_length_mins))#Remove any NA values(players who had no sessions or time played)

#Select only needed columns for final dataset
#Include mean_session_length, played_hours, number_of_sessions, experience
players_final_data<- players_merged|>
select(mean_session_length_mins, played_hours, number_of_sessions, experience, subscribe)
players_final_data 

In [ ]:
#Visualizations

#Bar graph 
Bar graph of number of sessions
sessions_num_graph<- players_final_data





In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
Scatterplot1<- players_final_data|>
ggplot(aes(x= mean_session_length_mins, y= number_of_sessions, color= subscribe, shape=subscribe))+
    geom_point(size=3)+ 
    labs(x= "Mean Session Time Played (in Minutes)", 
         y= "Number of Sessions Played", 
         title = "Relationship between Number of Sessions and Mean Session Time \nand their potential effect on Subscription",
         color="Subscription to Game Newsletter",
         shape="Subscription to Game Newsletter")+
    scale_color_brewer(palette = "Set2")+
    theme(text = element_text(size = 18))

Scatterplot1

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
histogram_1<- players_final_data|>
    ggplot(aes(x= played_hours, fill= as_factor(subscribe)))+
    geom_histogram(binwidth= 40, alpha = 0.8, position = "identity")+
     facet_grid(rows = vars(subscribe)) +
    labs(x="Number of Hours Played by Players",
         y="Player Count",
        title= "Relationship between Time Spent Playing \nand Subscription to the Game Newsletter")+
    theme(text = element_text(size = 18))
histogram_1

There is no visable relationship between the number of hours played and the mean session time.

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 8)
Scatterplot2<- players_final_data|>
ggplot(aes(x= number_of_sessions, y= played_hours, color= experience, shape=experience))+
    geom_point(size=3)+
    labs(x= "Number of Sessions Played", 
         y= "Total Number of Hours Played", 
         title = "Relationship between Number of Hours, number of sessions and experience",
         color="Subscription to Game Newsletter",
         shape="Subscription to Game Newsletter")+
    scale_color_brewer(palette = "Set2")+
    theme(text = element_text(size = 18))

Scatterplot2

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
bar_graph_1<- players_final_data|>
    ggplot(aes(x=Experience, y= played_hours)
    geom_bar(stat=identity)
    labs(x="Players Self Reported Experience Level",
         y="Number of Hours Played")
        title="Number of Hours Played and Experience Level 
    theme(text = element_text(size = 20))

In [ ]:
#Histogram 
options(repr.plot.width = 10, repr.plot.height = 10)
ggplot(aes(x= )
    geom_histogram(binwidth = , fill= )
    xlab("Number of 
    ylab(
    ggtitle(
    theme(text = element_text(size = 20))

In [ ]:
#Scatterplot of number of sessions and mean session length time 

In [ ]:
players_na<- colSums(is.na(sessions_time_mutate))
players_na

In [ ]:
str(players)

In [ ]:
#Number of sessions per player
player_sessions<- sessions |>
    group_by(hashedEmail)|> 
    summarize(number_of_sessions = n(), na.rm=TRUE)




In [ ]:

Data Wrangling:

Libraries:


library(repr)
library(ggplot2)
source("cleanup.R")

Tidy data:


Mean value for each quantitative variable:

mean_hours_played<- 

mean_age<- 


Visualizations:

